In [1]:
import face_recognition
import face_recognition.api as face_api
import os
import sys
import cv2
import numpy as np
import math
from twilio.rest import Client
import json
from urllib.request import urlopen


# Helper
def face_confidence(face_distance, face_match_threshold=0.5):
    range = (1.0 - face_match_threshold)
    linear_val = (1.0 - face_distance) / (range * 2.0)

    if face_distance > face_match_threshold:
        return str(round(linear_val * 100, 2)) + '%'
    else:
        value = (linear_val + ((1.0 - linear_val) * math.pow((linear_val - 0.5) * 2, 0.2))) * 100
        return str(round(value, 2)) + '%'


class FaceRecognition:
    def __init__(self):
        self.face_locations = []
        self.face_encodings = []
        self.face_names = []
        self.known_face_encodings = []
        self.known_face_names = []
        self.process_current_frame = True
        self.alerted_names = set()  # Prevent repeated alerts
        self.encode_faces()

    def encode_faces(self):
        print("[INFO] Encoding known faces...")
        for image in os.listdir('Datasets'):
            if image.lower().endswith(('.jpg', '.jpeg', '.png')):
                path = os.path.join('Datasets', image)
                img = face_recognition.load_image_file(path)
                encodings = face_recognition.face_encodings(img)
                if encodings:
                    self.known_face_encodings.append(encodings[0])
                    self.known_face_names.append(image)
                else:
                    print(f"[Warning] No face found in {image}, skipping.")

    def run_recognition(self):
        print("[INFO] Starting webcam...")
        video_capture = cv2.VideoCapture(0)

        if not video_capture.isOpened():
            sys.exit('Video source not found...')

        while True:
            ret, frame = video_capture.read()
            if not ret:
                print("[ERROR] Failed to grab frame")
                break

            if self.process_current_frame:
                # Resize frame of video to 1/4 size for faster processing
                small_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)
                # Convert from BGR (OpenCV) to RGB (face_recognition)
                rgb_small_frame = cv2.cvtColor(small_frame, cv2.COLOR_BGR2RGB)

                # Detect face locations
                self.face_locations = face_recognition.face_locations(rgb_small_frame)
                self.face_encodings = []

                # Compute face encodings manually with landmark detection and descriptor computation
                for face_location in self.face_locations:
                    try:
                        raw_landmarks = face_api._raw_face_landmarks(rgb_small_frame, [face_location])
                        if len(raw_landmarks) == 1:
                            face_descriptor = face_api.face_encoder.compute_face_descriptor(
                                rgb_small_frame,
                                raw_landmarks[0],
                                num_jitters=1
                            )
                            encoding = np.array(face_descriptor)
                            self.face_encodings.append(encoding)
                        else:
                            print("[WARNING] Landmarks not found correctly, skipping face.")
                    except Exception as e:
                        print("[ERROR] Skipping face due to:", e)

                self.face_names = []
                for face_encoding in self.face_encodings:
                    matches = face_recognition.compare_faces(self.known_face_encodings, face_encoding)
                    name = "Unknown"
                    confidence = '???'

                    face_distances = face_recognition.face_distance(self.known_face_encodings, face_encoding)
                    best_match_index = np.argmin(face_distances)

                    if matches[best_match_index]:
                        name = self.known_face_names[best_match_index]
                        confidence = face_confidence(face_distances[best_match_index])

                    label = f'{name} ({confidence})'
                    self.face_names.append(label)

                    # Alert logic
                    try:
                        conf_float = float(confidence.replace('%', ''))
                    except:
                        conf_float = 0

                    if conf_float >= 85 and name not in self.alerted_names:
                        self.alerted_names.add(name)

                        if name == "ganesh_c1.jpg":
                            print("✅ Recognized: Ganesh")
                            with open("ganesh_c1.txt", "r") as f:
                                print(f.read())
                                
                        elif name == "Dawood_ibrahim.jpg":
                            print("✅ Recognized: Dawood_ibrahim")
                            with open("Dawood_Ibrahim.txt", "r") as f:
                                print(f.read())

                        elif name == "Hafiz_Mohammed_Saeed.jpg":
                            print("✅ Recognized: Hafiz_Mohammed_Saeed")
                            with open("Hafiz Mohammed Saeed.txt", "r") as f:
                                print(f.read())

                        # Fetch IP info and send SMS alert
                        try:
                            url = 'http://ipinfo.io/json'
                            response = urlopen(url)
                            data = json.load(response)
                            print("[LOCATION DATA]", data)

                            # Twilio SMS
                            account_sid = 'AC124accf8811edf9e51b8145eeb115a75'
                            auth_token = '4d5ac9f00bb4d6500e9ad666c1fa1e4d'
                            client = Client(account_sid, auth_token)

                            message = client.messages.create(
                                body='Alert: Recognized Known Face!',
                                from_='+15739660126',
                                to='+918074936488'
                            )
                            print("[SMS Sent] SID:", message.sid)
                        except Exception as e:
                            print("[ERROR] SMS or IP Fetch Failed:", e)

            self.process_current_frame = not self.process_current_frame

            # Draw rectangles and labels
            for (top, right, bottom, left), name in zip(self.face_locations, self.face_names):
                top *= 4
                right *= 4
                bottom *= 4
                left *= 4

                cv2.rectangle(frame, (left, top), (right, bottom), (0, 255, 0), 2)
                cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 255, 0), cv2.FILLED)
                cv2.putText(frame, name, (left + 6, bottom - 6),
                            cv2.FONT_HERSHEY_DUPLEX, 0.8, (255, 255, 255), 1)

            cv2.imshow('Criminal Identification Using Face Recognition', frame)

            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

        video_capture.release()
        cv2.destroyAllWindows()


if __name__ == '__main__':
    fr = FaceRecognition()
    fr.run_recognition()


[INFO] Encoding known faces...
[INFO] Starting webcam...
✅ Recognized: Ganesh
Name     : Narsingu Ganesh
Age      : 18+
Dob      : 20-03-2002
Address  : kamareddygudem,thripuraram(mandal),nalgonda(dist),telangana-508207
crime    : murder

[ERROR] SMS or IP Fetch Failed: HTTP Error 429: Too Many Requests
✅ Recognized: Hafiz_Mohammed_Saeed

Name       : Hafiz Mohammed Saeed
DOB        : 1950-06-05
Address    : Sargodha, Punjab, Pakistan
Nationality:  Pakistani
crime      : The big boss of the Lashkar-e-Tayiba, 
	     Saeed is the main person behind the 26/11 bloodbath in Mumbai.
[ERROR] SMS or IP Fetch Failed: HTTP Error 429: Too Many Requests


In [3]:
!python --version


Python 3.10.18


In [ ]:
pip install twilio


In [ ]:
!pip install face_recognition opencv-python numpy twilio

In [ ]:
!python  --version

In [ ]:
pip install face_recognition


In [ ]:
pip install twilio


In [ ]:
import json
from urllib.request import urlopen
import os
from twilio.rest import Client

In [ ]:
url='http://ipinfo.io/json'
response=urlopen(url)
data=json.load(response)
print(data)
# Find your Account SID and Auth Token at twilio.com/console
# and set the environment variables. See http://twil.io/secure
account_sid = 'AC124accf8811edf9e51b8145eeb115a75'
auth_token = '4d5ac9f00bb4d6500e9ad666c1fa1e4d'
client = Client(account_sid, auth_token)

message = client.messages \
    .create(
         body='Alert!!!!',
         from_='+15739660126',
         to='+916303902129'
     )

In [ ]:
pip install geopy

In [ ]:
from geopy.geocoders import Nominatim
loc=Nominatim(user_agent="GetLoc")
getLoc=loc.geocode("india")
print(getLoc.address)
print("latitude",getLoc.latitude,"\n")
print("logitude",getLoc.longitude)

In [ ]:
pip install urllib3

In [ ]:
import json
from urllib.request import urlopen
url="http://ipinfo.io/json"
response=urlopen(url)
data=json.load(response)
print(data)
